In [1]:
import sys;
if sys.version != '3.6.2 |Continuum Analytics, Inc.| (default, Jul 20 2017, 13:51:32) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]':
    print('Warning: using an unexpected kernel')

In [2]:
import desc.imsim as imsim

In [3]:
%pylab inline
import seaborn as sns;sns.set_style('darkgrid')
%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams['figure.figsize'] = 19,14
sns.set_context('poster',font_scale=1.4)

Populating the interactive namespace from numpy and matplotlib


In [30]:
from lsst.sims.catalogs.definitions import InstanceCatalog
import lsst.sims.GalSimInterface.galSimCatalogs as cats
import lsst.sims.catUtils.exampleCatalogDefinitions as exampleCats
from lsst.sims.catalogs.db import CatalogDBObject
import lsst.sims.catUtils.baseCatalogModels as baseCatalogModels
from lsst.sims.catUtils.baseCatalogModels import *
from lsst.sims.catUtils.exampleCatalogDefinitions import *
from lsst.sims.utils import ObservationMetaData
from lsst.sims.catUtils.exampleCatalogDefinitions import ObsStarCatalogBase
from lsst.sims.catUtils.baseCatalogModels.StarModels import StarObj
from lsst.sims.catUtils.exampleCatalogDefinitions import ObsStarCatalogBase
psf = imsim.imSim.Kolmogorov_and_Gaussian_PSF()
from lsst.sims.catUtils.mixins import AstrometryStars, CameraCoordsLSST, ObsMetaMixin, PhotometryStars, VariabilityStars

### Using default reference star catalogs

In [33]:
db = CatalogDBObject.from_objid('allstars', database='LSSTCATSIM', host='fatboy.phys.washington.edu', port=1433, driver='mssql+pymssql')

In [45]:
# observation meta data restricts the catalog to a very small area on the sky so it doesn't take a lot of time for now.
cat = db.getCatalog('ref_catalog_star', ObservationMetaData(boundType='circle', boundLength=0.02, pointingDec=0, pointingRA=0, mjd=52000.))

In [41]:
cat.write_catalog('reference_cat.txt', chunk_size=10)

In [57]:
imsim.ImageSimulator('reference_cat.txt', psf=psf)

PhosimInstanceCatalogParseError: 
Required commands that are missing from the instance catalog reference_cat.txt:
   rightascension
   sunalt
   azimuth
   declination
   filter
   obshistid
   dist2moon
   nsnap
   vistime
   rottelpos
   moondec
   moonalt
   seed
   moonra
   altitude
   seeing
   mjd
   moonphase
   rotskypos

### Writing a class to check if I can get (one of) the missing info from the above catalog
Full list in error message

In [53]:
class myCat(InstanceCatalog, AstrometryStars, CameraCoordsLSST, PhotometryStars, VariabilityStars): 
# inheriting from a bunch of classes just in case one of them can provide a way to calculate the sunalt
        column_outputs = ['sunalt']

In [54]:
cat = myCat(db, ObservationMetaData(boundType='circle', boundLength=0.02, pointingDec=0, pointingRA=0, mjd=52000.))

ValueError: Required columns missing from database: (sunalt)

In [42]:
!more reference_cat.txt

uniqueId, objId, id, meanRaJ2000, meanDecJ2000, umag, gmag, rmag, imag, zmag, ym
ag
1125290210308, 1098916221, 1098916221, 0.00764670, 0.00011560, 29.19084640, 27.0
9697290, 26.11636920, 25.71587420, 25.53324020, 25.42696620
1195375732740, 1167359114, 1167359114, 0.00240610, 0.00844200, 18.43611590, 17.5
9754980, 17.29636530, 17.19141780, 17.16234610, 17.15884150
1125291210756, 1098917198, 1098917198, 0.01583110, 0.00862440, 29.22162850, 26.8
8769100, 25.62834040, 25.05813290, 24.77984400, 24.62647390
1134779251716, 1108182863, 1108182863, 359.99434240, 0.01709800, 31.69304000, 29
.15745000, 27.92010000, 26.83725000, 26.33434000, 26.11016000
1013810062340, 990048889, 990048889, 359.98859340, 0.01590100, 31.69513000, 29.8
3290000, 28.21852000, 25.62055000, 24.20004000, 23.02735000
1125289922564, 1098915940, 1098915940, 359.99602460, 0.01885590, 27.10841860, 25
.24763410, 24.41322380, 24.08392740, 23.93605040, 23.84466260
1125289024516, 1098915063, 1098915063, 359.98736380, 0.01390880, 2